In [11]:
import numpy as np
from collections import Counter, defaultdict
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

vocab=set(test_sentence)
vocab_len=len(vocab)
word_2_idx={word:i for i,word in enumerate(vocab) }
def build_cooccurence_matrix(corpus,window_size=4):
    cooccurrence_matrix= defaultdict(Counter)

    for i, word in enumerate(corpus):
        word_id=word_2_idx[word]
        context_start=max(0,i-window_size)
        context_end=min(i+window_size+1,len(corpus))

        for j in range(context_start,context_end):
            if i==j:
                continue
            context_word_id=word_2_idx[corpus[j]]
            cooccurrence_matrix[word_id][context_word_id] +=1/abs(i-j)
    
    return cooccurrence_matrix

cooccurrence_matrix = build_cooccurence_matrix(test_sentence)
def weighting_fn(x,x_max=100,alpha=0.75):
    return (x/x_max)**alpha if x<x_max else 1.0





In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

class GloVe(nn.Module):
    def __init__(self,vocab_size,embedding_size):
        super().__init__()
        self.w_embeddings=nn.Embedding(vocab_size,embedding_size) # target word embedding
        self.w_context_embeddings=nn.Embedding(vocab_size,embedding_size)
        self.w_biases=nn.Embedding(vocab_size,1)
        self.w_context_biases=nn.Embedding(vocab_size,1)

        nn.init.xavier_uniform_(self.w_embeddings.weight)
        nn.init.xavier_uniform_(self.w_context_embeddings.weight)
        nn.init.zeros_(self.w_biases.weight)
        nn.init.zeros_(self.w_context_biases.weight)
    
    def forward(self,word_i,word_j):
        w_i=self.w_embeddings(word_i)
        w_j=self.w_context_embeddings(word_j)
        b_i=self.w_biases(word_i).squeeze()
        b_j=self.w_biases(word_j).squeeze()
    
        dot_prod=torch.sum(w_i*w_j,dim=1)
        return dot_prod + b_i + b_j




def glove_loss_fn(prediction,occurence_value,x_max=100,alpha=0.75):
    weight=weighting_fn(occurence_value,x_max,alpha)

    loss=weight*torch.pow(prediction-torch.log(occurence_value+1e-10),2)
    return torch.sum(loss)

embedding_dim=100
window_size = 2
x_max = 100
alpha = 0.75
learning_rate = 0.05
epochs = 100

glove_model = GloVe(vocab_len, embedding_dim)
optimizer=optim.Adagrad(glove_model.parameters(),lr=learning_rate)
for epoch in range(epochs):
    total_loss=0
    for target_idx in cooccurrence_matrix:
        for context_idx,cooccurrence_value in cooccurrence_matrix[target_idx].items():
            target_tensor=torch.tensor([target_idx],dtype=torch.long)

            context_tensor=torch.tensor([context_idx],dtype=torch.long)
            cooccurrence_tensor = torch.tensor([cooccurrence_value], dtype=torch.float32)

            optimizer.zero_grad()
            prediction = glove_model(target_tensor, context_tensor)
            loss = glove_loss_fn(prediction, cooccurrence_tensor, x_max, alpha)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")


def get_word_embeddings(word,model,word_2_idx):
    word_idx = word_2_idx[word]
    embedding = model.w_embeddings(torch.tensor([word_idx], dtype=torch.long)).squeeze().detach()
    return embedding

word_embedding = get_word_embeddings("beauty", glove_model, word_2_idx)
print(f"Word embedding for 'beauty': {word_embedding}")

        




Epoch 10/100, Loss: 0.0004
Epoch 20/100, Loss: 0.0000
Epoch 30/100, Loss: 0.0000
Epoch 40/100, Loss: 0.0000
Epoch 50/100, Loss: 0.0000
Epoch 60/100, Loss: 0.0000
Epoch 70/100, Loss: 0.0000
Epoch 80/100, Loss: 0.0000
Epoch 90/100, Loss: 0.0000
Epoch 100/100, Loss: 0.0000
Word embedding for 'beauty': tensor([ 0.1443,  0.0599, -0.2380, -0.0123, -0.0466, -0.0273,  0.0719,  0.1271,
         0.0818,  0.0343,  0.1878, -0.1296,  0.1650,  0.0820,  0.0853, -0.0959,
        -0.0023,  0.2963,  0.2468,  0.1661,  0.0536,  0.0201,  0.2612,  0.0735,
        -0.0041,  0.0526, -0.0450, -0.1029,  0.1581,  0.2195,  0.1235,  0.0350,
         0.1897, -0.0354,  0.0235, -0.1574, -0.1211,  0.0015,  0.0576,  0.0852,
        -0.0614,  0.2721,  0.0233, -0.2112, -0.0892, -0.0615, -0.1444, -0.0472,
         0.0497,  0.0145,  0.0489,  0.1892, -0.1578, -0.0395,  0.2012, -0.1190,
        -0.1800, -0.1768,  0.0596,  0.0664,  0.0477, -0.0689, -0.0219, -0.0041,
        -0.0724,  0.0964, -0.0744,  0.1960,  0.1727, -0.1784